In [149]:
from scipy.stats import fisher_exact
fisher_exact([[1000, 2999],[12000, 2400]])

(0.06668889629876626, 0.0)

In [113]:
%matplotlib inline

import warnings
import os
from pathlib import Path
import quandl
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import graphviz
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, export_graphviz, _tree
from sklearn.linear_model import LinearRegression, Ridge, LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import roc_auc_score, roc_curve, mean_squared_error, precision_recall_curve
from sklearn.preprocessing import Imputer
import statsmodels.api as sm
from scipy.interpolate import interp1d, interp2d

In [114]:
warnings.filterwarnings('ignore')
plt.style.use('ggplot')

## Get Data

In [115]:
with pd.HDFStore('../data/assets.h5') as store:
    print(store.info())
    prices = store['quandl/wiki/prices'].adj_close.unstack('ticker')
    stocks = store['us_equities/stocks']

<class 'pandas.io.pytables.HDFStore'>
File path: ../data/assets.h5
/fred/assets                   frame        (shape->[4826,5])     
/quandl/wiki/prices            frame        (shape->[15389314,12])
/quandl/wiki/stocks            frame        (shape->[1,2])        
/sp500/prices                  frame        (shape->[37721,5])    
/sp500/stocks                  frame        (shape->[1,7])        
/us_equities/stocks            frame        (shape->[1,6])        


In [116]:
shared = prices.columns.intersection(stocks.index)
prices = prices.loc['2010': '2018', shared]
stocks = stocks.loc[shared, ['marketcap', 'ipoyear', 'sector']]

In [117]:
stocks.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2436 entries, A to ZUMZ
Data columns (total 3 columns):
marketcap    2432 non-null object
ipoyear      1074 non-null float64
sector       2396 non-null object
dtypes: float64(1), object(2)
memory usage: 76.1+ KB


In [118]:
prices.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2113 entries, 2010-01-04 to 2018-03-27
Columns: 2420 entries, A to ZUMZ
dtypes: float64(2420)
memory usage: 39.0 MB


### Create monthly return series

Remove outliers

In [119]:
returns = prices.resample('M').pct_change().stack().swaplevel()
returns = returns[returns.between(left=returns.quantile(.05), right=returns.quantile(.95))].to_frame('returns')

In [120]:
for t in range(1, 13):
    returns[f't-{t}'] = returns.groupby(level='ticker').returns.shift(t)
returns = returns.dropna()

In [121]:
# returns = returns.reset_index('date')
dates = returns.index.get_level_values('date')
returns['year'] = dates.year
returns['month'] = dates.month
returns = pd.get_dummies(returns, columns=['year', 'month'])

In [122]:
returns.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 171169 entries, (FCFS, 2010-08-31 00:00:00) to (ZUMZ, 2018-03-31 00:00:00)
Data columns (total 34 columns):
returns      171169 non-null float64
t-1          171169 non-null float64
t-2          171169 non-null float64
t-3          171169 non-null float64
t-4          171169 non-null float64
t-5          171169 non-null float64
t-6          171169 non-null float64
t-7          171169 non-null float64
t-8          171169 non-null float64
t-9          171169 non-null float64
t-10         171169 non-null float64
t-11         171169 non-null float64
t-12         171169 non-null float64
year_2010    171169 non-null uint8
year_2011    171169 non-null uint8
year_2012    171169 non-null uint8
year_2013    171169 non-null uint8
year_2014    171169 non-null uint8
year_2015    171169 non-null uint8
year_2016    171169 non-null uint8
year_2017    171169 non-null uint8
year_2018    171169 non-null uint8
month_1      171169 non-null uint8
month_2    

### Get stock characteristics

#### Create age proxy

In [123]:
stocks['age'] = pd.qcut(stocks.ipoyear, q=5, labels=list(range(1, 6))).astype(float).fillna(0).astype(int)
stocks = stocks.drop('ipoyear', axis=1)

#### Create size proxy

In [124]:
stocks.marketcap = stocks.marketcap.str.replace('$', '')
stocks['mcap'] = stocks.marketcap.str[-1]
stocks.marketcap =  pd.to_numeric(stocks.marketcap.str[:-1])
stocks = stocks[stocks.mcap.isin(['B', 'M'])]
stocks.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2431 entries, A to ZUMZ
Data columns (total 4 columns):
marketcap    2431 non-null float64
sector       2393 non-null object
age          2431 non-null int64
mcap         2431 non-null object
dtypes: float64(1), int64(1), object(2)
memory usage: 95.0+ KB


In [125]:
stocks.marketcap = stocks.apply(lambda x: x.marketcap * 1000 if x.mcap == 'B' else x.marketcap, axis=1)
stocks['size'] = pd.qcut(stocks.marketcap, q=10, labels=list(range(1, 11)))
stocks = stocks.drop(['mcap', 'marketcap'], axis=1)

#### Create Dummy variables

In [126]:
stocks.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2431 entries, A to ZUMZ
Data columns (total 3 columns):
sector    2393 non-null object
age       2431 non-null int64
size      2431 non-null category
dtypes: category(1), int64(1), object(1)
memory usage: 59.7+ KB


In [127]:
stocks = pd.get_dummies(stocks, 
                        columns=['size', 'age',  'sector'], 
                        prefix=['size', 'age', ''], 
                        prefix_sep=['_', '_', ''])
stocks.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2431 entries, A to ZUMZ
Data columns (total 28 columns):
size_1                   2431 non-null uint8
size_2                   2431 non-null uint8
size_3                   2431 non-null uint8
size_4                   2431 non-null uint8
size_5                   2431 non-null uint8
size_6                   2431 non-null uint8
size_7                   2431 non-null uint8
size_8                   2431 non-null uint8
size_9                   2431 non-null uint8
size_10                  2431 non-null uint8
age_0                    2431 non-null uint8
age_1                    2431 non-null uint8
age_2                    2431 non-null uint8
age_3                    2431 non-null uint8
age_4                    2431 non-null uint8
age_5                    2431 non-null uint8
Basic Industries         2431 non-null uint8
Capital Goods            2431 non-null uint8
Consumer Durables        2431 non-null uint8
Consumer Non-Durables    2431 non-null uint

### Combine data

In [134]:
data = (returns
        .reset_index('date')
        .merge(stocks, left_index=True, right_index=True)
        .dropna()
        .set_index('date', append=True))

s = len(returns.columns)
data.iloc[:, s:] = data.iloc[:, s:].astype(int).apply(pd.to_numeric, downcast='integer')
data.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 174551 entries, (A, 2011-02-28 00:00:00) to (ZUMZ, 2018-03-31 00:00:00)
Data columns (total 62 columns):
returns                  174551 non-null float64
t-1                      174551 non-null float64
t-2                      174551 non-null float64
t-3                      174551 non-null float64
t-4                      174551 non-null float64
t-5                      174551 non-null float64
t-6                      174551 non-null float64
t-7                      174551 non-null float64
t-8                      174551 non-null float64
t-9                      174551 non-null float64
t-10                     174551 non-null float64
t-11                     174551 non-null float64
t-12                     174551 non-null float64
year_2010                174551 non-null uint8
year_2011                174551 non-null uint8
year_2012                174551 non-null uint8
year_2013                174551 non-null uint8
year_2014           

### Create train-test indexes

### Store data

In [136]:
with pd.HDFStore('data.h5') as store:
    store.put('data', data)

In [138]:
with pd.HDFStore('data.h5') as store:
    data = store['data']
    data = data.drop([f't-{t}' for t in range(7, 13)] + [c for c in data.columns if c.startswith('month')], axis=1)
data.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 174551 entries, (A, 2011-02-28 00:00:00) to (ZUMZ, 2018-03-31 00:00:00)
Data columns (total 44 columns):
returns                  174551 non-null float64
t-1                      174551 non-null float64
t-2                      174551 non-null float64
t-3                      174551 non-null float64
t-4                      174551 non-null float64
t-5                      174551 non-null float64
t-6                      174551 non-null float64
year_2010                174551 non-null uint8
year_2011                174551 non-null uint8
year_2012                174551 non-null uint8
year_2013                174551 non-null uint8
year_2014                174551 non-null uint8
year_2015                174551 non-null uint8
year_2016                174551 non-null uint8
year_2017                174551 non-null uint8
year_2018                174551 non-null uint8
size_1                   174551 non-null int8
size_2                   174551 n

### Stock Prices

In [139]:
y = data.returns
X = data.drop('returns', axis=1)

## Explore Data

In [140]:
y.describe(percentiles=np.arange(.1, .91, .1))

count    174551.000000
mean          0.009913
std           0.055606
min          -0.128560
10%          -0.064080
20%          -0.036451
30.0%        -0.017798
40%          -0.002855
50%           0.010522
60%           0.023904
70%           0.038457
80%           0.056050
90%           0.082772
max           0.146335
Name: returns, dtype: float64

In [141]:
y_binary = (y>0).astype(int)

## Test KFold

In [142]:
class OneStepTimeSeriesSplit:
    """Generates tuples of train_idx, test_idx pairs
    Assumes the index contains a level labeled 'date'"""
    def __init__(self, n_splits=3):
        self.n_splits = n_splits

    def split(self, X, y=None, groups=None):
        dates = np.sort(X.index.get_level_values('date').unique())[::-1]
        X['idx'] = range(len(X))
        for date in dates[:self.n_splits]:
            train_idx = X.loc[X.index.get_level_values('date') < date, 'idx'].values
            test_idx = X.loc[X.index.get_level_values( 'date') == date, 'idx'].values
            yield train_idx, test_idx

    def get_n_splits(self, X, y, groups=None):
        return self.n_splits

### Manual split

In [143]:
kf = OneStepTimeSeriesSplit(n_splits=10)

In [144]:
for train, test in kf.split(X):
    print(X.iloc[test].index.get_level_values('date').unique())

DatetimeIndex(['2018-03-31'], dtype='datetime64[ns]', name='date', freq=None)
DatetimeIndex(['2018-02-28'], dtype='datetime64[ns]', name='date', freq=None)
DatetimeIndex(['2018-01-31'], dtype='datetime64[ns]', name='date', freq=None)
DatetimeIndex(['2017-12-31'], dtype='datetime64[ns]', name='date', freq=None)
DatetimeIndex(['2017-11-30'], dtype='datetime64[ns]', name='date', freq=None)
DatetimeIndex(['2017-10-31'], dtype='datetime64[ns]', name='date', freq=None)
DatetimeIndex(['2017-09-30'], dtype='datetime64[ns]', name='date', freq=None)
DatetimeIndex(['2017-08-31'], dtype='datetime64[ns]', name='date', freq=None)
DatetimeIndex(['2017-07-31'], dtype='datetime64[ns]', name='date', freq=None)
DatetimeIndex(['2017-06-30'], dtype='datetime64[ns]', name='date', freq=None)


### GridSearchCV

In [145]:
reg_tree = DecisionTreeRegressor(random_state=42)

param_grid = {'max_depth': range(1, 20, 2),
              'min_samples_leaf': [250, 500],
              'max_features': [None, .8, 'auto', 'sqrt']
              }

In [146]:
kf = GroupedTimeSeriesSplit(5)

NameError: name 'GroupedTimeSeriesSplit' is not defined

In [147]:
gridsearch_reg = GridSearchCV(estimator=reg_tree,
                              param_grid=param_grid,
                              scoring='neg_mean_squared_error',
                              n_jobs=-1,
                              cv=kf,
                              refit=True,
                              return_train_score=True)

In [148]:
gridsearch_reg.fit(X=X, y=y)

KeyboardInterrupt: 

In [ ]:
gridsearch_reg.best_params_